In [ ]:
from itertools import chain
import numpy as np
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from matplotlib import cm
from neuron import h
import json
import pickle
from dlutils import utils
from dlutils.cell import Cell, branch_order
from dlutils.synapse import AMPANMDAExp2Synapse
from dlutils.spine import Spine
%matplotlib inline

### General parameters

In [ ]:
cell_type = 'a-thorny'
optimization_folder = '/Users/daniele/Postdoc/Research/Janelia/01_model_optimization/'

if cell_type == 'thorny':
    base_folder = optimization_folder + 'Thorny/DH070813/20191208071008_DH070813_/'
    swc_file = 'DH070813-.Edit.scaled.converted.swc'
    cell_name = 'DH070813_'
    individual = 1
else:
    base_folder = optimization_folder + 'A-thorny/DH070213C3/20191206232623_DH070213C3_/'
    swc_file = 'DH070213C3-.Edit.scaled.converted.swc'
    cell_name = 'DH070213C3_'
    individual = 0
    
swc_file = base_folder + swc_file
params_file = base_folder + 'individual_{}.json'.format(individual)
config_file = base_folder + 'parameters.json'

passive = False
with_TTX = True
replace_axon = True
add_axon_if_missing = True
parameters = json.load(open(params_file, 'r'))
mechanisms = utils.extract_mechanisms(config_file, cell_name)
sim_pars = pickle.load(open(base_folder + 'simulation_parameters.pkl','rb'))
replace_axon = sim_pars['replace_axon']
add_axon_if_missing = not sim_pars['no_add_axon']

### Instantiate the cell

In [ ]:
cell = Cell('CA3_cell_%d' % int(np.random.uniform()*1e5), swc_file, parameters, mechanisms)
cell.instantiate(replace_axon, add_axon_if_missing, force_passive=passive, TTX=with_TTX)
if cell_type == 'thorny':
#     section_num = 0
#     section_num = 10
    section_num = 14
else:
#     section_num = 0
    section_num = 16

section = cell.morpho.apic[section_num]
print('Branch order of section {}: {}.'.format(section.name(), branch_order(section)))
# Ra = 150
Ra = section.Ra

### Instantiate the spines

In [ ]:
# in the Harnett paper, the head is spherical with a diameter of 0.5 um: a cylinder
# with diameter and length equal to 0.5 has the same (outer) surface area as the sphere
head_L = 0.5         # [um]
head_diam = 0.5      # [um]
neck_L = 1.58        # [um]
neck_diam = 0.077    # [um]
spine_distance = 5   # [um] distance between neighboring spines
n_spines = 9
L = spine_distance * (n_spines - 1)
norm_L = L / section.L
if section_num == 0:
    spines = [Spine(section, x, head_L, head_diam, neck_L, neck_diam, Ra, i) \
              for i,x in enumerate(np.linspace(0.5 - norm_L/2, 0.5 + norm_L/2, n_spines))]
else:
    spines = [Spine(section, x, head_L, head_diam, neck_L, neck_diam, Ra, i) \
              for i,x in enumerate(np.linspace(0.2 - norm_L/2, 0.2 + norm_L/2, n_spines))]

for spine in spines:
    spine.instantiate()

In [ ]:
segments = [section(spine._sec_x) for spine in spines]
flag = True
for i in range(n_spines - 1):
    if segments[i] == segments[i+1]:
        flag = False
        print('Spines #{} and #{} are connected to the same segment.'.format(i, i+1))
if flag:
    print('All spines are connected to different segments on the dendritic branch.')

#### Show where the spines are located on the dendritic tree

In [ ]:
plt.figure(figsize=(10,10))
for sec in chain(cell.morpho.apic, cell.morpho.basal):
    if sec in cell.morpho.apic:
        color = 'k'
    else:
        color = 'b'
    lbl = sec.name().split('.')[1].split('[')[1][:-1]
    n = sec.n3d()
    sec_coords = np.zeros((n,2))
    for i in range(n):
        sec_coords[i,:] = np.array([sec.x3d(i), sec.y3d(i)])
    middle = int(n / 2)
    plt.text(sec_coords[middle,0], sec_coords[middle,1], lbl, \
             fontsize=14, color='m')
    plt.plot(sec_coords[:,0], sec_coords[:,1], color, lw=1)
for spine in spines:
    plt.plot(spine._points[:,0], spine._points[:,1], 'r.')
# plt.axis([-100,0,0,200])
plt.axis('equal');

### Insert a synapse into each spine

In [ ]:
E = 0        # [mV]

if cell_type == 'thorny':
#     data = pickle.load(open('DH070813_1_with_TTX_expt_type_TTX_Ra=1x_apical[10](0.5).pkl', 'rb'))
#     weight = data['weight']
    if section_num == 0:
        weights = np.array([0.001, 1 * 0.004])  # AMPA and NMDA weights
    elif section_num == 10:
        weights = np.array([0.0001, 1 * 0.0005])  # AMPA and NMDA weights
    elif section_num == 14:
        weights = np.array([0.0007, 1 * 0.005])  # AMPA and NMDA weights
    else:
        weights = None
    
    AMPA_taus = {'tau1': 0.5, 'tau2':  5.0}
    NMDA_taus = {'tau1': 1.0, 'tau2': 50.0}

else:
#     data = pickle.load(open('DH070213C3_0_with_TTX_expt_type_CTRL_Ra=1x_apical[0](0.5).pkl', 'rb'))
#     weight = data['weight']
    if section_num == 0:
        weights = np.array([0.001, 1 * 0.004])  # AMPA and NMDA weights
    elif section_num == 16:
        weights = np.array([0.0003, 1 * 0.00015])  # AMPA and NMDA weights
    else:
        weights = None

    AMPA_taus = {'tau1': 0.5, 'tau2':   5.0}
    NMDA_taus = {'tau1': 1.0, 'tau2': 100.0}

# AMPA_taus = {k: v for k,v in data['AMPA'].items()}
# NMDA_taus = {k: v for k,v in data['NMDA'].items()}
# AMPA_NMDA_ratio = data['AMPA_NMDA_ratio']
# AMPA_NMDA_ratio = 2

print('AMPA:')
print('    tau_rise = {:.3f} ms'.format(AMPA_taus['tau1']))
print('   tau_decay = {:.3f} ms'.format(AMPA_taus['tau2']))
print('NMDA:')
print('    tau_rise = {:.3f} ms'.format(NMDA_taus['tau1']))
print('   tau_decay = {:.3f} ms'.format(NMDA_taus['tau2']))
# print('AMPA/NMDA ratio = {:.3f}'.format(AMPA_NMDA_ratio))

synapses = [AMPANMDAExp2Synapse(spine.head, 1, E, weights, AMPA = AMPA_taus, \
                                NMDA = NMDA_taus) for spine in spines]

for syn in synapses:
    syn.nmda_syn.alpha_vspom *= 2
    syn.nmda_syn.v0_block -= 20
    syn.nmda_syn.eta *= 0.1
print('       alpha = {:.3f} 1/mV'.format(synapses[0].nmda_syn.alpha_vspom))
print('    v0_block = {:.3f} mV'.format(synapses[0].nmda_syn.v0_block))
print('         eta = {:.3f}'.format(synapses[0].nmda_syn.eta))

#### Functions used to describe the removal of the Mg block from the NMDA synapse

In [ ]:
extMgConc = 1
alpha_vspom = -0.062
v0_block = 10
eta = 0.2801
vspom = lambda v: 1. / (1. + eta * extMgConc * np.exp(alpha_vspom * (v - v0_block)))

extMgConc2 = synapses[0].nmda_syn.extMgConc
alpha_vspom2 = synapses[0].nmda_syn.alpha_vspom
v0_block2 = synapses[0].nmda_syn.v0_block
eta2 = synapses[0].nmda_syn.eta
vspom2 = lambda v: 1. / (1. + eta2 * extMgConc2 * np.exp(alpha_vspom2 * (v - v0_block2)))

v = np.arange(-100, 50)
sigmoid = lambda v: 1. / (1. + np.exp(- (v + 40)))
vharnett = lambda v: 1. / (1. + np.exp(-0.062 * v * extMgConc / 3.57))

plt.figure(figsize=(8,4))
plt.plot(v, vharnett(v), 'k', label='Harnett')
plt.plot(v, vspom(v), 'r', label='Maex & De Schutter')
plt.plot(v, vspom2(v), 'm', label='Modified Maex & De Schutter')
plt.plot(v, sigmoid(v), 'g', label='Sigmoid')
plt.xlabel('Voltage (mV)')
plt.ylabel('Mg unblock')
plt.legend(loc='best')

#### Compute the presynaptic spike times

In [ ]:
t0 = 1000.
dt = 1000.
spike_times = [np.sort(t0 + (n_spines - 1) * dt - np.arange(i) * dt) for i in range(n_spines, 0, -1)]
poisson = False
sequential = True
if sequential:
    for i in range(n_spines):
        for j in range(n_spines-i):
            spike_times[i][j] += i * 0.3
elif poisson:
    # frequency of the incoming spikes
    F = 1 / 0.3e-3
    for i in range(n_spines):
        ISI = - np.log(np.random.uniform(size=n_spines*2)) / F
        spks = np.cumsum(ISI)
        for j in range(n_spines - i):
            spike_times[j][i] += spks[j]

for syn, spks in zip(synapses, spike_times):
    syn.set_presynaptic_spike_times(spks)

In [ ]:
spike_times

In [ ]:
rec = {}
for lbl in 't','Vsoma':
    rec[lbl] = h.Vector()
rec['t'].record(h._ref_t)
rec['Vsoma'].record(cell.morpho.soma[0](0.5)._ref_v)
for i,spine in enumerate(spines):
    rec['Vapic-{}'.format(i)] = h.Vector()
    rec['Vhead-{}'.format(i)] = h.Vector()
    rec['IAMPA-{}'.format(i)] = h.Vector()
    rec['gAMPA-{}'.format(i)] = h.Vector()
    rec['INMDA-{}'.format(i)] = h.Vector()
    rec['gNMDA-{}'.format(i)] = h.Vector()
    rec['MgBlock-{}'.format(i)] = h.Vector()
    rec['Vapic-{}'.format(i)].record(spine._sec(spine._sec_x)._ref_v)
    rec['Vhead-{}'.format(i)].record(spine.head(0.5)._ref_v)
    rec['IAMPA-{}'.format(i)].record(synapses[i].syn[0]._ref_i)
    rec['gAMPA-{}'.format(i)].record(synapses[i].syn[0]._ref_g)
    rec['INMDA-{}'.format(i)].record(synapses[i].syn[1]._ref_i)
    rec['gNMDA-{}'.format(i)].record(synapses[i].syn[1]._ref_g)
    rec['MgBlock-{}'.format(i)].record(synapses[i].syn[1]._ref_mgBlock)

h.cvode_active(1)
h.tstop = t0 + n_spines * dt
h.run()

In [ ]:
t = np.array(rec['t'])
iampa = np.array([np.array(rec['IAMPA-{}'.format(i)])*1e3 for i in range(n_spines)])
inmda = np.array([np.array(rec['INMDA-{}'.format(i)])*1e3 for i in range(n_spines)])
gampa = np.array([np.array(rec['gAMPA-{}'.format(i)])*1e3 for i in range(n_spines)])
gnmda = np.array([np.array(rec['gNMDA-{}'.format(i)])*1e3 for i in range(n_spines)])
MgBlock = np.array([np.array(rec['MgBlock-{}'.format(i)])for i in range(n_spines)])
vhead = np.array([np.array(rec['Vhead-{}'.format(i)]) for i in range(n_spines)])
vapic = np.array([np.array(rec['Vapic-{}'.format(i)]) for i in range(n_spines)])

In [ ]:
idx, = np.where((t > 990) & (t < 1200))
EPSP_spine = np.max(vhead[0,idx]) - vhead[0,-1]
EPSP_dend = np.max(vapic[0,idx]) - vapic[0,-1]
AR = EPSP_spine / EPSP_dend
print('AR = {:.3f}'.format(AR))
plt.plot(t[idx], vhead[0,idx], 'k', label='Spine')
plt.plot(t[idx], vapic[0,idx], 'r', label='Dendrite')
plt.xlabel('Time (ms)')
plt.ylabel('Vm (mV)')
plt.legend(loc='best');

In [ ]:
k = 6
i = k
idx, = np.where((t > k*dt + 990) & (t < k*dt + 1200))
fig,(ax1,ax2) = plt.subplots(1, 2, figsize=(12,5))
ax1.plot(t[idx], gnmda[i,idx], 'g', label=r'$\mathrm{G}_{NMDA}$ w/o Mg block')
ax1.plot(t[idx], gnmda[i,idx] * MgBlock[i,idx], 'm', label=r'$\mathrm{G}_{NMDA}$')
ax1.plot(t[idx], gampa[i,idx], 'k', label=r'$\mathrm{G}_{AMPA}$')
ax1.set_xlabel('Time (ms)')
ax1.set_ylabel('Conductance (nS)')
ax1.legend(loc='best')
ax2.plot(t[idx], vhead[k,idx], 'k')
ax2.plot(t[idx], vapic[k,idx], color=[.6,.6,.6])
# ax2.plot(t[idx], vhead[i,idx], 'r')
# ax2.plot(t[idx], vhead[i,idx], color=[1,.6,.6])

In [ ]:
i = 6
idx, = np.where((t > i*dt + 1000 - 10) & (t < i*dt + 1020))
cmap = cm.get_cmap('rainbow', n_spines)
fig,ax = plt.subplots(3, 2, figsize=(12,16))
for j,(va,vh,gN,iN,gA,iA,Mg) in enumerate(zip(vapic, vhead, gnmda, inmda, gampa, iampa, MgBlock)):
    ax[0,0].plot(t[idx], va[idx], color=cmap(j))
    ax[0,1].plot(t[idx], vh[idx], color=cmap(j), label='Spine {}'.format(j+1))
    ax[1,0].plot(t[idx], gN[idx], color=cmap(j))
    ax[1,1].plot(t[idx], Mg[idx], color=cmap(j))
    #ax[1,1].plot(t[idx], iN[idx], color=cmap(j), label='Spine {}'.format(j+1))
    ax[2,0].plot(t[idx], gA[idx], color=cmap(j))
    ax[2,1].plot(t[idx], iA[idx], color=cmap(j), label='Spine {}'.format(j+1))
#     ax[1,0].plot(t[idx] - j*0.5, gN[idx] * Mg[idx], color=cmap(j))
#     ax[1,1].plot(t[idx] - j*0.5, iN[idx], color=cmap(j), label='Spine {}'.format(j+1))
#     ax[2,0].plot(t[idx] - j*0.5, gA[idx], color=cmap(j))
#     ax[2,1].plot(t[idx] - j*0.5, iA[idx], color=cmap(j), label='Spine {}'.format(j+1))
for i in range(3):
    for j in range(2):
        ax[i,j].set_xlabel('Time (ms)')
ax[0,0].set_ylabel('Dendritic voltage (mV)')
ax[0,1].set_ylabel('Spine head voltage (mV)')
ax[1,0].set_ylabel('NMDA conductance (nS)')
ax[1,1].set_ylabel('NMDA current (pA)')
ax[2,0].set_ylabel('AMPA conductance (nS)')
ax[2,1].set_ylabel('AMPA current (pA)')
ax[0,1].legend(loc='best')

In [ ]:
# fig,ax = plt.subplots(n_spines, n_spines, figsize=(n_spines*2,n_spines*2))
# for i in range(n_spines):
#     tstart = t0 + i*dt - 10
#     tstop = tstart + 210
#     idx, = np.where((t > tstart) & (t < tstop))
#     for j in range(n_spines):
#         ax[j,i].plot(t[idx], iampa[j,idx], 'm')
#         ax[j,i].plot(t[idx], inmda[j,idx], 'g')
#         ax[j,i].set_xticklabels([])
#         ax[j,i].set_yticklabels([])
#         if i == 0:
#             ax[j,i].set_ylabel('Spine #{}'.format(j+1))
#         if j == n_spines - 1:
#             ax[j,i].set_xlabel('Event #{}'.format(i+1))

In [ ]:
# fig,ax = plt.subplots(n_spines, n_spines, figsize=(n_spines*2,n_spines*2))
# for i in range(n_spines):
#     tstart = t0 + i*dt - 10
#     tstop = tstart + 210
#     idx, = np.where((t > tstart) & (t < tstop))
#     for j in range(n_spines):
#         ax[j,i].plot(t[idx], vhead[j,idx], 'm')
#         ax[j,i].plot(t[idx], vapic[j,idx], 'g')
#         ax[j,i].set_ylim([-70, -30])
#         ax[j,i].set_xticklabels([])
#         ax[j,i].set_yticklabels([])
#         if i == 0:
#             ax[j,i].set_ylabel('Spine #{}'.format(j+1))
#         if j == n_spines - 1:
#             ax[j,i].set_xlabel('Event #{}'.format(i+1))
# # plt.savefig('tmp.pdf')

In [ ]:
# fig,ax = plt.subplots(3, 1, figsize=(6,15), sharex=True)

# # xlim = [470,np.max(spike_times)+150]
# xlim = [t0-100,h.tstop]

# t = np.array(rec['t'])
# idx, = np.where((t > xlim[0]) & (t < xlim[1]))
# step = int(np.floor(256 / n_spines))
# for i in range(n_spines):
#     col = cm.jet(i*step)
#     vhead = np.array(rec['Vhead-{}'.format(i)])
#     vdend = np.array(rec['Vapic-{}'.format(i)])
#     ax[0].plot(t[idx], vhead[idx], color=col, label='Spine #{}'.format(i+1))
# #     ax[0].plot(spike_times[i] + np.zeros(2), [-68,-66], color=col, lw=2)
#     ax[1].plot(t[idx], vdend[idx], color=col, label='Spine #{}'.format(i+1))
# #     ax[1].plot(spike_times[i] + np.zeros(2), [-68,-66], color=col, lw=2)
# #ax[1].plot(rec['t'], rec['Vsoma'], 'k', label='Soma')
# vsoma = np.array(rec['Vsoma'])
# ax[2].plot(t[idx], vsoma[idx], 'k', label='Soma')

# ax[2].set_xlabel('Time (ms)')
# ax[0].set_ylabel('Spine voltage (mV)')
# ax[1].set_ylabel('Dendritic voltage (mV)')
# ax[2].set_ylabel('Somatic voltage (mV)')
# # ax1.legend(loc='best')
# # ax2.legend(loc='best')
# # ax[0].set_xlim()
# ax[0].set_xlim([8900,9200])

In [ ]:
# idx, = np.where(t > t0*0.95)
k = 0
# x = t[idx]
# y = vapic[k][idx]
v_pks = []
g_pks = []
# fig,(ax1,ax2) = plt.subplots(1, 2, figsize=(12,5))
fig,(ax1,ax2,ax3) = plt.subplots(1, 3, figsize=(16,5))
for spk in spike_times[0]:
    idx, = np.where((t > spk - 10) & (t < spk + 50))
    v_pks.append(np.argmax(vapic[k,idx]) + idx[0])
    g_pks.append(np.argmax(gnmda[k,idx] * MgBlock[k,idx]) + idx[0])
    ax1.plot(t[idx] - spk, vapic[k,idx], 'k')
    ax1.plot(t[v_pks[-1]] - spk, vapic[k,v_pks[-1]], 'r.')
    ax2.plot(t[idx] - spk, vhead[k,idx], 'k')
    ax3.plot(t[idx] - spk, gnmda[k,idx] * MgBlock[k,idx], 'm')
ax1.set_xlabel('Time (ms)')
ax1.set_ylabel('Dendritic Vm (mV)')
ax2.set_xlabel('Time (ms)')
ax2.set_ylabel('Spine Vm (mV)')

outfile = '{}dendritic_Vm_section_{}_'.format(cell_name, section_num)

if synapses[0].nc[1].weight[0] > 0:
    plt.savefig(outfile + 'with_NMDA.pdf')
else:
    plt.savefig(outfile + 'without_NMDA.pdf')

In [ ]:
dV = vapic[k,v_pks] - vapic[k,-1]
dG = gnmda[k,g_pks] * MgBlock[k,g_pks]

fig,(ax1,ax2) = plt.subplots(1, 2, figsize=(12,5))
ax1.plot(1 + np.arange(n_spines), dV, 'ko-', markerfacecolor='w', markersize=7, linewidth=1)
# ax1.plot(1 + np.arange(n_spines), dV[0] * np.arange(1, n_spines+1), 'ro-', \
#          markerfacecolor='w', markersize=7, linewidth=1)
ax1.set_xticks(1 + np.arange(n_spines))
ax1.set_xlabel('Input number')
ax1.set_ylabel('Dendritic EPSP (mV)')
ax2.plot(1 + np.arange(n_spines), dG, 'ko-', markerfacecolor='w', markersize=7, linewidth=1)
ax2.set_xticks(1 + np.arange(n_spines))
ax2.set_xlabel('Input number')
ax2.set_ylabel('NMDA conductance (nS)')

outfile = '{}dV_section_{}_'.format(cell_name, section_num)
if synapses[0].nc[1].weight[0] > 0:
    pickle.dump({'dV': dV, 'dG': dG}, open(outfile + 'with_NMDA.pkl', 'wb'))
    plt.savefig(outfile + 'with_NMDA.pdf')
else:
    pickle.dump({'dV': dV, 'dG': dG}, open(outfile + 'without_NMDA.pkl', 'wb'))
    plt.savefig(outfile + 'without_NMDA.pdf')

In [ ]:
if cell_type == 'thorny':
    col = np.array([0,0,0])
else:
    col = np.array([1,0,0])

fig,(ax1,ax2) = plt.subplots(1, 2, figsize=(12,5))
for i,lbl in enumerate(['with', 'without']):
    pkl_file = '{}dV_section_{}_{}_NMDA.pkl'.format(cell_name, section_num, lbl)
    data = pickle.load(open(pkl_file, 'rb'))
    dV = data['dV']
    dG = data['dG']
    n_spines = len(dV)
    c = np.min([np.ones(3), col + i*0.6], axis=0)
    ax1.plot(1 + np.arange(n_spines), dV, 'o-', label=lbl + ' NMDA', color=c, markerfacecolor='w', \
             markersize=7, linewidth=1)
    if lbl == 'with':
        ax2.plot(1 + np.arange(n_spines), dG, 'o-', label=lbl + ' NMDA', color=c, markerfacecolor='w', \
                 markersize=7, linewidth=1)
ax1.set_xlabel('Input number')
ax1.set_ylabel('Dendritic EPSP (mV)')
ax1.legend(loc='best')
ax2.set_xlabel('Input number')
ax2.set_ylabel('NMDA conductance (nS)')

plt.savefig('{}dV_section_{}.pdf'.format(cell_name, section_num))